In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import sympy as sp

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
m1, m2, m3 = 1.0, 1.0, 1.0  # массы
l1, l2 = 1.0, 1.0  # длины звеньев
a1, a2, a3 = 0.5, 0.5, 0.5  # положение CoM
I1, I2, I3 = 0.1, 0.1, 0.1  # моменты инерции
g = 9.81  # ускорение свободного падения

B1 = m1*a1**2 + I1 + m2*l1**2 + m3*l1**2
B2 = m2*a2**2 + I2 + m3*l2**2
B3 = m2*a2*l1 + m3*l1*l2
B4 = m3*a3**2 + I3
B5 = m3*a3*l1

B6 = m3*a3*l2

In [ ]:
def M(q1, q2, q3):
  M11 = B1 + B2 + 2*B3*np.cos(q2) + 2*B5*np.cos(q2 - q3) + 2*B6*np.cos(q3)
  M12 = B2 + B3*np.cos(q2) + B4 + B5*np.cos(-q2 + q3) + 2*B6*np.cos(q3)
  M13 = -B4 - B5*np.cos(-q2 + q3) - B6*np.cos(q3)
  
  M21 = M12
  M22 = B2 + B4 + 2*B6*np.cos(q3)
  M23 = -B4 - B6*np.cos(q3)

  M31 = M13
  M32 = M23
  M33 = B4

  M = np.array([[M11, M12, M13],
                [M21, M22, M23],
                [M31, M32, M33]])

  return M